# Préparation et modélisation des données

In [1]:
# Import des bibliothèques
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import pandas as pd  

# Lecture CSV

df = pd.read_csv('diabetes.csv')

# Affichage des 3 premières lignes
df.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


Les features de chaque individu sont représentées par les huit premiers attributs du dataset et la variable à prédire est 'outcome'.

In [5]:
# Séparation des données et de la cible
data = df.drop('Outcome', axis=1)
target = df['Outcome']



# Séparation des données

In [6]:
# Décomposition des données
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=4)

# Voting Classifier

Le voting Classifier est un méta-classifier qui combine plusieurs modèles, parfois similaires mais souvent conceptuellement différents par vote majoritaire. 

Pour la suite  nous allons créer trois classifieurs radicalement différents, afin d’essayer d’obtenir le meilleur de chacun.

Nous allons créer les trois classifieurs suivants nommés respectivement clf1, clf2 et clf3 :
- Un modèle de classe KNeighborsClassifier, avec 3 voisins (n_neighbors=3)
- Un modèle RandomForestClassifier, avec le paramètre random_state=123
- Un modèle LogisticRegression, avec le paramètre max_iter=1000 pour assurer la convergence du modèle.


In [7]:
clf1 = KNeighborsClassifier(n_neighbors=3)
clf2 = RandomForestClassifier(random_state=123)
clf3 = LogisticRegression(max_iter=1000)

La construction d’un voting classifier se fait à l’aide d’une instance de la classe VotingClassifier qui prend en paramètres :
-	Estimators : une liste contenant pour chaque estimateur un label et le nom de l’estimateur
-	Voting : permet de spécifier la méthode de vote (hard ou soft)


Pour rappel : 
-	Avec le vote hard : l’étiquette de la classe finale est celle de la classe prédite le plus fréquemment par les modèles de classification. 
-	Avec le vote soft : l’étiquette de la classe finale est obtenue en faisant la moyenne des probabilités de classe.


In [8]:
vclf = VotingClassifier(estimators=[('knn', clf1), ('rf', clf2), ('lr', clf3)], voting = 'hard')

-	A l’aide de la fonction Kfold, création de cv3 : un cross-validator à 3 parties (folds), avec les paramètres random_state=111 et shuffle = True
-	A l’ide de la fonction cross_validate(), affichage pour chacun des 3 classifieurs individuels ainsi que pour le voting classifier :
o	La moyenne et l’écart type du score de taux de bonne prédiction(‘accuracy’)
o	La moyenne et l’écart type du f1-score obtenus par validation croisée avec cv3 sur (X_train, y_train)


In [9]:
cv3 = KFold(n_splits=3, random_state=111, shuffle=True)

for clf, label in zip([clf1, clf2, clf3, vclf], ['KNN', 'Random Forest', 'Logic Regression', 'Voting Classifier']):
    scores = cross_validate(clf, X_train, y_train, cv=cv3, scoring=['accuracy', 'f1'])
    print("[%s] \n Accuracy : %0.2f (+/- %0.2f)"% (label, scores['test_accuracy'].mean(), scores['test_accuracy'].std()),
          "F1 score : %0.2f (+/- %0.2f)"%(scores['test_f1'].mean(), scores['test_f1'].std()))

[KNN] 
 Accuracy : 0.71 (+/- 0.02) F1 score : 0.58 (+/- 0.02)
[Random Forest] 
 Accuracy : 0.76 (+/- 0.03) F1 score : 0.63 (+/- 0.04)
[Logic Regression] 
 Accuracy : 0.76 (+/- 0.02) F1 score : 0.61 (+/- 0.05)
[Voting Classifier] 
 Accuracy : 0.77 (+/- 0.00) F1 score : 0.64 (+/- 0.02)


Les résultats obtenus montrent que sur nos deux métriques, le Voting Classifier fonctionne sensiblement mieux que le meilleur des modèles individuels.

En évaluant individuellement chaque trio de prédictions retournées par les classifieurs, le modèle final parvient à combiner le pouvoir des différents modèles pour produire de meilleurs résultats.


# Stacking

Le Stacking est une méthode qui utilise les sorties de différents classifieurs comme entrées d'un nouveau méta-classifieur, défini en amont.
Le modèle de Stacking avec scikit-learn est créé à partir de la classe StackingClassifier.
Il s'utilise de la même manière que le VotingClassifier, mais un paramètre supplémentaire, final_estimator, permet d'indiquer le modèle à utiliser comme méta-classifieur.


In [17]:
sclf = StackingClassifier(estimators=[('knn', clf1), ('rf', clf2), ('lr', clf3)], final_estimator=clf3)

scores = cross_validate(sclf, X_train, y_train, cv=cv3, scoring=['accuracy', 'f1'])
    
print("[StackingClassifier]: \n Accuracy: %0.2f (+/- %0.2f)\n" % (scores['test_accuracy'].mean(), scores['test_accuracy'].std()),
      "F1 score: %0.2f (+/- %0.2f)" % (scores['test_f1'].mean(), scores['test_f1'].std()))


[StackingClassifier]: 
 Accuracy: 0.76 (+/- 0.02)
 F1 score: 0.62 (+/- 0.04)


Les résultats de validation croisée sont bons et au moins égaux aux performances du meilleur classifieur individuel.

Évaluons maintenant nos deux modèles d'Ensemble sur le jeu de test.

In [18]:
vclf.fit(X_train, y_train)
sclf.fit(X_train, y_train)

print("Acc :", vclf.score(X_test, y_test))
print("Acc :", sclf.score(X_test, y_test))

Acc : 0.8138528138528138
Acc : 0.8008658008658008


Les résultats obtenus sur l'ensemble de test sont très satisfaisants, pour évaluer l'utilité de nos modèles, comparons-les avec le meilleur classifieur individuel selon les résultats obtenus plus haut.

In [19]:
clf2.fit(X_train, y_train)

print("Acc :", clf2.score(X_test, y_test))
## On peut également essayer de remplacer clf2 par clf1 et clf3 pour voir s'ils font mieux.


Acc : 0.7878787878787878


Au vu des résultats du modèle de RandomForest, nous pouvons conclure que nos modèles d'ensemble permettent de gagner ~0.02 points d'accuracy par rapport à l'utilisation d'un seul modèle.
Cet écart n'est pas négligeable, ce qui les rendent très intéressants dans les cas où la performance du modèle est très importante.

